In [1]:
import os
import openai
openai.api_key = ""

In [2]:
def tag_stage1(user_input, coding_problem):
    instruction = """Now is the question clarification stage of a technical interview. 
    You need to flag their question using the following schema, if the input is:
    Unrelated to solving the coding problem, answer "N1";
    Directly asking for solution or code samples, answer "N2";
    Not a question, answer "N3".
    Otherwise, answer "Y"."""
    
    conversation = [
        {"role": "system", "content": instruction},
        {"role": "assistant", "content": coding_problem},
        {"role": "user", "content": user_input}
    ]

    response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=conversation
    )
    tag = response['choices'][0]['message']['content']
    return tag

In [3]:
class Stage1_Question_Clarification:
    _instruction = """
    You are an interviewer. 
    This is the time for the interviewee to ask clarification questions during the interview.
    If the interviewee is not asking questions, ask probign questions to encourage them.
    Start with a broad question and gradually get more specific based on their responses. 
    For example, start with "Don't you have any questions?" and progress to "Do you understand what the input data type is?" 
    Remember to adapt your probing questions based on the programmer's responses. 
    If they ask a clarifying question, provide an answer. 
    Avoid being repetitive in your questions."""
    
    _coding_problem = """
    Here's the coding problem. 
    You are given two non-empty linked lists representing two non-negative integers. 
    The digits are stored in reverse order, and each of their nodes contains a single digit. 
    Add the two numbers and return the sum as a linked list. 
    You can only ask me clarification questions."""

    def __init__(self):
        self.conversation = [
        {"role": "system", "content": self._instruction},
        {"role": "assistant", "content": self._coding_problem}
        ]
    
    def answer(self, user_input):
        if tag_stage1(user_input, self._coding_problem) != 'Y':
            return "Please ask a question relevant to clarifying the coding problem."
        self.conversation.append({"role": "user", "content": user_input})
        response = openai.ChatCompletion.create(
          model="gpt-4",
          messages=self.conversation
        )
        answer = response['choices'][0]['message']['content']
        self.conversation.append({"role": "assistant", "content": answer})
        return answer

    def probe(self):
        response = openai.ChatCompletion.create(
          model="gpt-4",
          messages=self.conversation
        )
        question = response['choices'][0]['message']['content']
        self.conversation.append({"role": "assistant", "content": question})
        return question

In [4]:
! pip install ipywidgets

In [5]:
import ipywidgets as widgets
from IPython.display import display


In [6]:
session = Stage1_Question_Clarification()


In [7]:
user_input = widgets.Textarea(
    value='',
    placeholder='Type your question here...',
    description='User Input:',
    disable=False
)

probe_button = widgets.Button(
    description='Get a Probing Question',
    disable=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

answer_button = widgets.Button(
    description='Get an Answer',
    disable=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)


In [8]:
def probe(b):
    question = session.probe()
    print(f"Assistant: {question}")

def answer(b):
    question = user_input.value
    response = session.answer(question)
    print(f"Assistant: {response}")

In [9]:
probe_button.on_click(probe)
answer_button.on_click(answer)


In [10]:
display(user_input, probe_button, answer_button)


Textarea(value='', description='User Input:', placeholder='Type your question here...')

Button(description='Get a Probing Question', style=ButtonStyle(), tooltip='Click me')

Button(description='Get an Answer', style=ButtonStyle(), tooltip='Click me')

Assistant: Don't you have any questions?
Assistant: Please ask a question relevant to clarifying the coding problem.
Assistant: Do you understand what the input data type is?
